In [1]:
# 필수 라이브러리 선언
import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException

import time
import atexit
import os
import re


In [17]:
options = ChromeOptions()
browser = Chrome()
browser.get("https://scholar.google.co.kr/schhp?hl=ko")

In [18]:
query = browser.find_element_by_name("q")
query.clear()
query.send_keys("코로나19 관광")
query.send_keys(Keys.RETURN)

In [24]:

#총개수
import re

#https://regex101.com/r/ZB2N2o/1/
expression_kor = "<.*>.*[\t\r\n\v\f]*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*[\t\r\n\v\f]*<\/.*>" #한글이 포함된 태그 찾는 정규식 , 문제는 태그 안에 태그가 있으면 안에 있는 태그를 선택한다는 것~
expression_eng = "<.*>.*[\t\r\n\v\f]*[a-zA-Z]+.*[\t\r\n\v\f]*<\/.*>" 

pattern_kor = re.compile(expression_kor)
pattern_eng = re.compile(expression_eng)

current_page_index = 1

#구글 페이지의 끝을 알 수 없으니 while 문으로 변경 필요
article_data = []
article_colums = ['제목', '국문초록(예측)', '영문초록(예측)']

start_index = 0

for i in range(start_index, 10) :
    # 첫페이지는 이미 선택되었기 때문에 지나감
    if i > start_index :
        next_page = browser.find_element_by_xpath(f'//a[@class="gs_nma" and text()="{i + 1}"]')
        browser.execute_script("arguments[0].click()", next_page)
        time.sleep(2)

    title_count = len(browser.find_elements_by_xpath('//h3[@class="gs_rt"]/a'))
    for index in range(title_count):
        article = []

        time.sleep(2)
        title = browser.find_element_by_xpath(f'(//h3[@class="gs_rt"])[{index + 1}]/a')

        #타이틀은?
        title_text = title.get_attribute('innerHTML')
        title_text = re.sub('<.+?>', '', title_text, 0, re.I|re.S)

        title.click()
        time.sleep(2)
        
        all_html_tag = browser.find_element_by_tag_name("html")
        all_html = all_html_tag.get_attribute("innerHTML")
        
        tags_include_korean = pattern_kor.findall(all_html)
        tags_include_english = pattern_eng.findall(all_html)
        
        # #으로 시작하고 ;로 끝나는 문자도 정리 필요 한글 초록
        kor_values = [re.sub('<.+?>', '', sentence, 0, re.I|re.S) for sentence in tags_include_korean]
        kor_values_len = [len(re.sub('<.+?>', '', sentence, 0, re.I|re.S)) for sentence in tags_include_korean]

        eng_values = [re.sub('<.+?>', '', sentence, 0, re.I|re.S) for sentence in tags_include_english]
        eng_values_len = [len(re.sub('<.+?>', '', sentence, 0, re.I|re.S)) for sentence in tags_include_english]
        
        abstract_kor = "" if kor_values_len == 0 else kor_values[kor_values_len.index(max(kor_values_len))].strip()
        abstract_eng = "" if eng_values_len == 0 else eng_values[eng_values_len.index(max(eng_values_len))].strip()

        article = [title_text, abstract_kor, abstract_eng]

        print(f'[{i}-{index}] {article}')

        article_data.append(article)
        browser.back()

    current_page_index = current_page_index + 1

df = pd.DataFrame(article_data, columns=article_colums)
df.to_csv("./article.csv")

ination are more likely to be return migrants, taking advantage of kinship ties and location-specific amenities. We suspect that the expected difference in returns may result from a trade-off between the lower costs associated with making a return migration, the possibility of lower income levels following migration, and the help of kinship and location-specific capital in the home regions, relying upon relatives and friends for monetary or social assistance during the transition period right after migration.']
[5-1] ['의료 관광의 미래 전략 시나리오에 관한 연구', '본 연구에서는 의료 관광의 미래전략에 관한 시나리오 작성을 통해 의료 관광의 산업구조 변화를 전망하고 전략적 의미를 파악하고자 한다. 의료 관광의 산업구조를 의료서비스기관과 이들을 의료 관광객과 연결시켜 주 는 의료 관광 플랫폼으로 구분하고, 분야별 전문가 인터뷰 및 계층화 분석법(AH P)을 통해서 산업의 성장요인과 성장 저해 요인을 파악하여 발생 가능한 시나리오를 작성하였고, 각 시나리오별로 예상되는 전개 양상과 대응전략에 대한 시사점을 도출하였다.', 'This study examines the future strategy of medical tourism by making up scenarios, foresees the changes of medical toursim industry, and finds strategic implications. We classify medical s

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class="gs_nma" and text()="9"]"}
  (Session info: chrome=87.0.4280.88)


In [31]:
import pandas as pd
df = pd.DataFrame(article_data, columns=article_colums)
df.to_csv("./article.csv", index=False, encoding="utf-8-sig")